# SageMaker で提供される DeepAR アルゴリズムの学習と推論を行う

#### ノートブックに含まれる内容

- Amazon が提供するアルゴリズムの使いかた
- 中でも，DeepAR アルゴリズムの使い方

#### ノートブックで使われている手法の詳細

- アルゴリズム: DeepAR
- データ: ランダム時系列を作成
- 可視化手段: matplotlib


## Deep AR とは

DeepAR とは，AWS から提供されている state-of-the-art の時系列データ予測アルゴリズムです．このノートブックでは，DeepAR を使って，モデルの学習と推論を行います．DeepAR は，スカラの時系列データに対する予測を行う，教師あり学習アルゴリズムです．古典的な ARIMA モデルや指数平滑法は，単一時系列のみを学習に用いており，そのためしばしば極端な予測値を出力します．DeepAR は複数の時系列を入力として取り，併せて学習することで，より安定した予測結果を出すことができます．そのため，例えば多種の製品の需要，サーバ負荷，Webページへのリクエスト数といった複数の時系列が手に入るような場合に，非常に有効です．

DeepAR の詳細については，[公式ドキュメントの解説](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html)および [arXiv の論文](https://arxiv.org/abs/1704.04110)をご覧ください

## セットアップ

In [ ]:
import time
import numpy as np
np.random.seed(1)
import pandas as pd
import json
import matplotlib.pyplot as plt

ここでは SageMaker SDK を使って作業を行います．また，学習データを S3 にアップロードする際に，s3fs モジュールを使用します．当該モジュールがインスタンスに入っていないため，ここでは conda コマンドでインストールします（もちろん，その他のモジュールも `pip` コマンドでインストールすることができます）．

In [ ]:
!conda install -y s3fs

In [ ]:
import boto3
import s3fs
import sagemaker
from sagemaker import get_execution_role

データを置くバケットを指定します．このバケットは，ノートブックインスタンスがある（そして，学習と推論を行う）リージョンと同一である必要があります．また SageMaker セッションで使用する IAM role に，データに対するアクセス権限を与える必要があります．ここでは `get_execution_role` を使って，ノートブックインスタンス作成時に割り当てたロールを使用します．

以下の**<span style="color: red;"> 1 行目のバケット名について，`sagemaker-us-east-1-handson-YYYYMMDD-XX` の `YYYYMMDD` を今日の日付に，`XX` を指定された適切な数字に変更</span>**してください

In [ ]:
bucket = 'sagemaker-us-east-1-handson-YYYYMMDD-XX'
prefix = 'sagemaker/DEMO-deepar'

sagemaker_session = sagemaker.Session()
role = get_execution_role()

boto3.client('s3').create_bucket(Bucket=bucket)
s3_data_path = "{}/{}/data".format(bucket, prefix)
s3_output_path = "{}/{}/output".format(bucket, prefix)

次に，当該リージョンで提供されているコンテナイメージを取得します．AWS から提供されているイメージについては，[公式ドキュメントに一覧](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html)があります．

In [ ]:
containers = {
    'us-east-1': '522234722520.dkr.ecr.us-east-1.amazonaws.com/forecasting-deepar:latest',
    'us-east-2': '566113047672.dkr.ecr.us-east-2.amazonaws.com/forecasting-deepar:latest',
    'us-west-2': '156387875391.dkr.ecr.us-west-2.amazonaws.com/forecasting-deepar:latest',
    'eu-west-1': '224300973850.dkr.ecr.eu-west-1.amazonaws.com/forecasting-deepar:latest'
}
image_name = containers[boto3.Session().region_name]

## データのロード

ここでは，毎時単位のデータを用います．そのため，値を予測する範囲を指定する `prediction_length` は，今後 48 時間分のデータを表す 48 とします．

In [ ]:
freq = 'H'
prediction_length = 48

またこのモデルでは，モデルが直前何時点ぶんのデータを考慮して予測を行うかを指定する，`context_length` というパラメタを設定する必要があります．この値を決める際には，データの周期性を考慮する必要があります．例えば，毎時単位のデータであれば，24 時点の周期性を仮定することができます．そのほかにも週次，月次の周期性を改定することも可能です．

In [ ]:
context_length = 72

以下で，各 400 時点ぶんのデータを含んだ，200 個のノイズが含まれた，24 時間周期の時系列データを生成します．すべての時系列データは，`t0` 時点からスタートするものとします．

In [ ]:
t0 = '2016-01-01 00:00:00'
data_length = 400
num_ts = 200
period = 24

In [ ]:
time_series = []
for k in range(num_ts):
    level = 10 * np.random.rand()
    seas_amplitude = (0.1 + 0.3*np.random.rand()) * level
    sig = 0.05 * level # noise parameter (constant in time)
    time_ticks = np.array(range(data_length))
    source = level + seas_amplitude*np.sin(time_ticks*(2*np.pi)/period)
    noise = sig*np.random.randn(data_length)
    data = source + noise
    index = pd.DatetimeIndex(start=t0, freq=freq, periods=data_length)
    time_series.append(pd.Series(data=data, index=index))

In [ ]:
time_series[0].plot()
plt.show()

学習/テストデータの分割については，200 個の各時系列データについて，最後の 48 時点ぶんを除いた 352 時点ぶんのデータを学習データとします．テストデータは 400 時点すべてが含まれたものになります．

In [ ]:
time_series_training = []
for ts in time_series:
    time_series_training.append(ts[:-prediction_length])

In [ ]:
time_series[0].plot(label='test')
time_series_training[0].plot(label='train', ls=':')
plt.legend()
plt.show()

`pandas.Series` フォーマットの時系列データを，[Deep AR で指定している JSON 形式](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html)に変換して，S3 にアップロードします．

In [ ]:
def series_to_obj(ts, cat=None):
    obj = {"start": str(ts.index[0]), "target": list(ts)}
    if cat:
        obj["cat"] = cat
    return obj

def series_to_jsonline(ts, cat=None):
    return json.dumps(series_to_obj(ts, cat))

In [ ]:
encoding = "utf-8"
s3filesystem = s3fs.S3FileSystem()

with s3filesystem.open(s3_data_path + "/train/train.json", 'wb') as fp:
    for ts in time_series_training:
        fp.write(series_to_jsonline(ts).encode(encoding))
        fp.write('\n'.encode(encoding))

with s3filesystem.open(s3_data_path + "/test/test.json", 'wb') as fp:
    for ts in time_series:
        fp.write(series_to_jsonline(ts).encode(encoding))
        fp.write('\n'.encode(encoding))

## モデルの学習

まず，`Estimator` オブジェクトを作成します

In [ ]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.m4.xlarge',
    base_job_name='DEMO-deepar',
    output_path="s3://" + s3_output_path
)

作成した `Estimator` に対して，ハイパーパラメタをセットします．

In [ ]:
hyperparameters = {
    "time_freq": freq,
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "num_cells": "40",
    "num_layers": "3",
    "likelihood": "gaussian",
    "epochs": "20",
    "mini_batch_size": "32",
    "learning_rate": "0.001",
    "dropout_rate": "0.05",
    "early_stopping_patience": "10"
}

In [ ]:
estimator.set_hyperparameters(**hyperparameters)

学習に使用するデータを指定して，ジョブを開始します．この際に `test` データチャネルを指定することで，学習済みモデルをテストデータに適用した際の，accuracy を出力してくれます．

In [ ]:
data_channels = {
    "train": "s3://{}/train/".format(s3_data_path),
    "test": "s3://{}/test/".format(s3_data_path)
}

estimator.fit(inputs=data_channels)

## モデルの推論を実行

In [ ]:
job_name = estimator.latest_training_job.name

endpoint_name = sagemaker_session.endpoint_from_job(
    job_name=job_name,
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    deployment_image=image_name,
    role=role
)

推論の際に，`pandas.Series` オブジェクトを入力として渡せるようにするための，ラッパークラスを定義します．本来は DeepAR で指定された JSON フォーマットなためです．

In [ ]:
class DeepARPredictor(sagemaker.predictor.RealTimePredictor):

    def set_prediction_parameters(self, freq, prediction_length):
        """Set the time frequency and prediction length parameters. This method **must** be called
        before being able to use `predict`.
        
        Parameters:
        freq -- string indicating the time frequency
        prediction_length -- integer, number of predicted time points
        
        Return value: none.
        """
        self.freq = freq
        self.prediction_length = prediction_length
        
    def predict(self, ts, cat=None, encoding="utf-8", num_samples=100, quantiles=["0.1", "0.5", "0.9"]):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.
        
        Parameters:
        ts -- list of `pandas.Series` objects, the time series to predict
        cat -- list of integers (default: None)
        encoding -- string, encoding to use for the request (default: "utf-8")
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])
        
        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_times = [x.index[-1]+1 for x in ts]
        req = self.__encode_request(ts, cat, encoding, num_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, prediction_times, encoding)
    
    def __encode_request(self, ts, cat, encoding, num_samples, quantiles):
        instances = [series_to_obj(ts[k], cat[k] if cat else None) for k in range(len(ts))]
        configuration = {"num_samples": num_samples, "output_types": ["quantiles"], "quantiles": quantiles}
        http_request_data = {"instances": instances, "configuration": configuration}
        return json.dumps(http_request_data).encode(encoding)
    
    def __decode_response(self, response, prediction_times, encoding):
        response_data = json.loads(response.decode(encoding))
        list_of_df = []
        for k in range(len(prediction_times)):
            prediction_index = pd.DatetimeIndex(start=prediction_times[k], freq=self.freq, periods=self.prediction_length)
            list_of_df.append(pd.DataFrame(data=response_data['predictions'][k]['quantiles'], index=prediction_index))
        return list_of_df

In [ ]:
predictor = DeepARPredictor(
    endpoint=endpoint_name,
    sagemaker_session=sagemaker_session,
    content_type="application/json"
)
predictor.set_prediction_parameters(freq, prediction_length)

実際に予測を行なって，結果をグラフに表示します．

In [ ]:
# 訓練に使ってデータの，最初の 5 つの時系列を投げて，その先 48 時点ぶんの値を予測します
list_of_df = predictor.predict(time_series_training[:5])
actual_data = time_series[:5]

In [ ]:
for k in range(len(list_of_df)):
    plt.figure(figsize=(12,6))
    actual_data[k][-prediction_length-context_length:].plot(label='target')
    p10 = list_of_df[k]['0.1']
    p90 = list_of_df[k]['0.9']
    plt.fill_between(p10.index, p10, p90, color='y', alpha=0.5, label='80% confidence interval')
    list_of_df[k]['0.5'].plot(label='prediction median')
    plt.legend()
    plt.show()

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
sagemaker_session.delete_endpoint(endpoint_name)